Belief - Localization maintains a function over all possible places the road might be where each cell has an associated probability value
Sense is the measurement update function which is the normalized product of the probability values (beliefs) by a corresponding weight based on if they match the measurement or not
Motion - a convolution (for each possible location after the motion, we reverse engineer the situation and guess where the 'world' might have come from and add the corresponding probabilities)

Probability function 0<= P(X) <= 1
Summation of P(Xi) = 1
Xi is a grid cell in the world

Measurement update seeks to calculate a belief of my location after seeing the measurement P(X|Z)
X = grid cell 
Z = measurement
It takes our prior distribution, P(X), and tries to calculate the chances of seeing a red or green tile for every possible location P(Z|X)
The product of the prior of the grid cell P(Xi) * measurement probability which was large if the color measurement corresponded to the color in the grid cell and small if not, P(Z|Xi) gives us the non normalized posterior distribution of the grid cell, P(Xi|Z).
The normalizer is P(Z) which is the sum of P(Z|Xi)*P(Xi)
## Sensing
### Bayes Theorm
X = grid cells;
Z = measurement;
P(X|Z) = measurement probability which is large if Z matches with X and small if Z does not match with X

$$P(X|Z) =  \frac{P(Z|X)P(X)}{P(Z)} = \frac{P(Z|X)P(X)}{\sum P(Z|X_{i})P( X_{i})} $$
$$ \bar{P}(X_i|Z) = P(Z|X_i)P(X_i) $$
$$ \alpha = \sum \bar{p}(X_i|Z) $$


## Motion
### Total Probability/ Convolution
$$ P(X^t_i) = P(X^{t-i}_j).P(X_i|X_j) $$
$$ P(A) = \sum_b P(A|B)P(B) $$


In [9]:
p = [0.2,0.2,0.2,0.2,0.2]
# representing images of the surface of the world as a set of green and red patches
# color of lane markings relative to the pavement
# an entire field of observations
# camera image in model
world =["green","red","red","green","green"]
# representing camera images that the robot takes
# camera image in measurement
measurements=["red","green"]
motions = [1,1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

In [5]:
def sense(p,Z):
        q=[]
        for i in range(len(p)):
            #evaluates to 1 or 0 depending on what Z is and world[i] is
            hit = (Z == world[i])
            #sets either pHit or pMiss 0 based on the value of hit
            q.append(p[i]*(hit*pHit)+p[i]*((1-hit)*pMiss))
        s=sum(q)
        for i in range(len(q)):
            q[i] = q[i]/s
        return q


In [6]:
def move(p, U):
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1) % len(p)]
        s = s + pUndershoot * p[(i-U+1) % len(p)]
        q.append(s)
    return q

In [8]:

for k in range(len(measurements)):
    p = sense(p, measurements[k])
    p = move(p, motions[k])

print(p)

[0.21369651673066375, 0.29952345035655514, 0.11336738891936368, 0.13998902907295668, 0.23342361492046082]


In [10]:
#representing a 4x5 dimensional world with only 2 possible colors
colors = [['red', 'green', 'green', 'red', 'red',],
         ['red', 'red', 'green', 'red', 'red',],
         ['red', 'red', 'green', 'green', 'red',],
         ['red', 'red', 'red', 'red', 'red',]]

#measurement vector observing red twice
# NOTE: always need to have as many measurements as motions (below)
measurements = ['green', 'green', 'green', 'green', 'green']

#motion vector indicating first stay in place - [0,0], then move one step to the right [0,1]
#others are [1,0], for move down
#[0,1] for move right
#[-1, 0] move up

motion = [[0, 0], [0, 1],[1, 0], [1, 0],[0, 1]]

# probability that the sensor value is correct - in this case always correct 
# indicates a noise free sensor
sensor_right = 1.0
#At what probability the motion is executed correctly - now always correct. 
#smaller value indicates motion might fail in which case our robot wont move at all
p_move = 1.0

sensor_wrong = 1.0 - sensor_right
p_stay = 1.0 - p_move
